### 0. Импорты и инициализация

In [ ]:
import logging
import os

import mlflow
import numpy as np
import pandas as pd
import scipy
import pickle

from dotenv import load_dotenv
from mlflow import MlflowClient
from sklearn.preprocessing import LabelEncoder
from implicit.als import AlternatingLeastSquares

from utils.mlflow_shortcuts import create_unique_run_by_name, get_run_by_name
from utils.s3_shortcuts import check_s3_credentials_in_environment

In [2]:
from threadpoolctl import threadpool_limits

# Следуем рекомендациям модуля implicit для оптимизации ALS
threadpool_limits(1, "blas");

In [3]:
# Настройка форматов
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [4]:
# Загружаем переменные окружения
load_dotenv()

# Проверяем наличие необходимых переменных для доступа к S3
check_s3_credentials_in_environment()

# Проверяем наличие переменных для доступа к серверу mlfow
assert os.getenv('MLFLOW_HOST') and os.getenv('MLFLOW_PORT')

# Устанавливаем трекинг сервер
mlflow.set_tracking_uri(
    f"http://{os.getenv('MLFLOW_HOST')}:{os.getenv('MLFLOW_PORT')}"
)

# Устанавливаем уровень логгироания для mlflow
logging.getLogger('mlflow').setLevel(logging.WARNING)

In [5]:
RANDOM_STATE = 123

EXPERIMENT_NAME = 'Final_project'

RUN_NAME_GET_CAT_TREE = '1.1 ETL - category tree '
RUN_NAME_GET_ITEMS = '1.2 ETL - item_properties'
RUN_NAME_GET_EVENTS = '1.3 ETL - events'
RUN_NAME_TRAIN_TEST_SPLIT = '2. Train/test split'
RUN_NAME_TOP_POPULAR = '3.1 Top popular'
RUN_NAME_NAIVE_ALS = '3.2 Naive ALS'
RUN_NAME_WEIGHTED_ALS = '3.3 Weighted ALS'

# Пути к .csv файлам с исходными данными
PATH_CSV_CAT_TREE = '../data/category_tree.csv'
PATH_CSV_ITEMS1 = '../data/item_properties_part1.csv'
PATH_CSV_ITEMS2 = '../data/item_properties_part2.csv'
PATH_CSV_EVENTS = '../data/events.csv'

# Пути к предобработанным таблицам
PATH_CLEAN_CAT_TREE = '../data/cat_tree.parquet'
PATH_CLEAN_ITEMS = '../data/items.parquet'
PATH_CLEAN_EVENTS = '../data/events.parquet'

# Пути к обучающим/тестовым данным
PATH_EVENTS_TRAIN = '../data/events_trian.parquet'
PATH_EVENTS_TEST = '../data/events_test.parquet'
PATH_ITEMS_TRAIN = '../data/items_trian.parquet'

# Пути к результатам (моделям)
PATH_TOP_POPULAR = 'models/top_popular.parquet'
PATH_NAIVE_ALS = 'models/naive_als.parquet'
PATH_WEIGHTED_ALS = 'models/weighted_als.parquet'

### 1. Загрузка и предобработка данных

__Действия по очистке (фильтрации) даыннх по результатам EDA__

- из каталога товаров удаляем товары с item_id, отсутствующим в дереве категорий
- из всех признаков товаров, оставляем только category_id
- из таблицы событий удаляем события, относящиеся к 'неизвестынм' товарам (отсутствующим в каталоге items)

#### 1.1. Category tree ETL

In [6]:
# Загружаем данные из .csv и переименовываем столбцы
cat_tree = (
    pd.read_csv(PATH_CSV_CAT_TREE)
    .rename(columns={
        'categoryid': 'category_id',
        'parentid': 'parent_id'
    })
)

cat_tree.head(3)

,category_id,parent_id
0,1016,213.00
1,809,169.00
2,570,9.00


In [7]:
# Проверка на отсутствие дубликатов category_id
assert cat_tree['category_id'].duplicated().sum() == 0

In [8]:
# Добалвяем столбец 'parents' с перечнем родителей
cat_tree['parents'] = pd.NA

# Получим содержимое таблицы в виде списка кортежей (category_id, parent_id)
categories = list(cat_tree[['category_id', 'parent_id']]
                  .itertuples(index=False))

# Временно поменяем индекс таблицы на 'category_id'
cat_tree.set_index('category_id', inplace=True)

# Итерируем, пока в списке кортежей есть элементы (необработанне категории)
while (categories_left := len(categories)):

    # Проходим по копии (!) списка необработанных категорий
    for category_id, parent_id in list(categories):
        
        # Если категория не является категорией верхнего уровня
        # И ее родительская категория еще не обработана (не размечена в таблице)
        # переходим к следующей
        if (
            pd.notna(parent_id) 
            and parent_id not in (
               cat_tree[cat_tree['parents'].notna()].index
            )
        ):
            continue

        # Для всех категорий, кроме верхнего уровня, список родителей это:
        # родители родителя плюс сам родитель :)
        # А для верхнего уровня список родителей пустой.
        cat_tree.at[category_id, 'parents'] = (
            cat_tree.at[parent_id, 'parents'] + [int(parent_id)]
            if pd.notna(parent_id) else []
        )
            
        # Удаляем категорию из списка необработанных 
        categories.pop(categories.index((category_id, parent_id)))

    # Если после прохода число категорий в списке не уменьшилось
    # то прерываем цикл (битые данные: категории с неизвестными parent_id) 
    if len(categories) == categories_left:
        break

# Возвращаем 'category_id' обратно в столбцы 
cat_tree.reset_index(inplace=True)

# Перепакуем списки в кортежи
cat_tree['parents'] = cat_tree['parents'].map(
    lambda x: tuple(x) if isinstance(x, list) else x
)

# Проверяем, что обработали все категории
assert len(categories) == 0

# Визуальная проверка результата
cat_tree.head(3)

,category_id,parent_id,parents
0,1016,213.00,"(1532, 1299, 213)"
1,809,169.00,"(395, 1257, 169)"
2,570,9.00,"(653, 351, 9)"


In [9]:
# Добавляем признак категории верхнего уровня
cat_tree['top_cat_id'] = (
    cat_tree[['category_id', 'parents']]
    .apply(lambda x: x.parents[0] if x.parents else x.category_id, axis=1)
)

# Визуальная проверка
cat_tree.tail(3)

,category_id,parent_id,parents,top_cat_id
1666,1336,745.00,"(140, 540, 745)",140
1667,689,207.00,"(1490, 207)",1490
1668,761,395.00,"(395,)",395


In [10]:
# Сохраняем обработанную таблицу локально
cat_tree.to_parquet(PATH_CLEAN_CAT_TREE)

In [11]:
# Логируем артефакты шага
with mlflow.start_run(
    run_id=create_unique_run_by_name(EXPERIMENT_NAME, RUN_NAME_GET_CAT_TREE)
):
    mlflow.log_artifact(PATH_CLEAN_CAT_TREE)

#### 1.2. Items ETL

In [12]:
# Загружаем данные из таблиц, переименовываем столбцы
items = pd.concat([
    pd.read_csv(PATH_CSV_ITEMS1),
    pd.read_csv(PATH_CSV_ITEMS2)
]).rename(columns={'itemid': 'item_id'})

items.head(3)

,timestamp,item_id,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566


In [13]:
# Из всех значений 'property' оставляем только 'categoryid',
# приводим 'value' к типу int
items = (
    items[items['property'] == 'categoryid']
    .astype(dtype={'value': 'int'})
)

# Приводим 'timestamp' к типу datetime
items['timestamp'] = pd.to_datetime(
    items['timestamp'], unit='ms'
)

# Визуальная проверка
items.head(3)

,timestamp,item_id,property,value
0,2015-06-28 03:00:00,460429,categoryid,1338
140,2015-05-24 03:00:00,281245,categoryid,1277
151,2015-06-28 03:00:00,35575,categoryid,1059


In [14]:
# Удалим столбец 'property', столбец 'value' переименуем в 'category_id'
items = (
    items
    .drop(columns='property')
    .rename(columns={'value': 'category_id'})
    # Отсортируем таблицу
    .sort_values(by=['item_id', 'timestamp'], ignore_index=True)
)

# Визуальная проверка
items.head(3)

,timestamp,item_id,category_id
0,2015-05-10 03:00:00,0,209
1,2015-05-10 03:00:00,1,1114
2,2015-05-10 03:00:00,2,1305


In [15]:
# Загрузим подготовленную таблицу с деревом категорий
cat_tree = pd.read_parquet(PATH_CLEAN_CAT_TREE)

cat_tree.head(3)

,category_id,parent_id,parents,top_cat_id
0,1016,213.00,"[1532, 1299, 213]",1532
1,809,169.00,"[395, 1257, 169]",395
2,570,9.00,"[653, 351, 9]",653


In [16]:
# Добавим в таблицу items признаки 'parents', 'top_cat_id'
items = items.merge(
    cat_tree[['category_id', 'parents', 'top_cat_id']],
    on='category_id',
    how='inner' 
).reset_index(drop=True)

items.head(3)

,timestamp,item_id,category_id,parents,top_cat_id
0,2015-05-10 03:00:00,0,209,"[1532, 293]",1532
1,2015-05-10 03:00:00,1,1114,"[1532, 113]",1532
2,2015-05-10 03:00:00,2,1305,"[653, 1202, 1214]",653


In [17]:
# Сохраним обработанную таблицу локально
items.to_parquet(PATH_CLEAN_ITEMS)

In [18]:
# Логируем артефакты шага
with mlflow.start_run(
    run_id=create_unique_run_by_name(EXPERIMENT_NAME, RUN_NAME_GET_ITEMS)
):
    mlflow.log_artifact(PATH_CLEAN_ITEMS)

#### 1.3. Events ETL

In [19]:
# Загружаем данные из таблицы, переименовываем столбцы и сортируем таблицу
events = (
    pd.read_csv(PATH_CSV_EVENTS)
    .rename(columns={
        'visitorid': 'user_id',
        'itemid': 'item_id',
        'transactionid': 'transaction_id'
    })
    .sort_values(by='timestamp', ignore_index=True)
)

# Приведем timestamp к типу datetime
events['timestamp'] = pd.to_datetime(events['timestamp'], unit='ms')

events.head(3)

,timestamp,user_id,event,item_id,transaction_id
0,2015-05-03 03:00:04.384,693516,addtocart,297662,NaN
1,2015-05-03 03:00:11.289,829044,view,60987,NaN
2,2015-05-03 03:00:13.048,652699,view,252860,NaN


In [20]:
# Удалим из таблицы событий товары, отсутствующие в каталоге items
events = events[
    events['item_id'].isin(items['item_id'].drop_duplicates())
].reset_index(drop=True)

In [21]:
# Сохраняем обработанную таблицу локально
events.to_parquet(PATH_CLEAN_EVENTS)

In [22]:
# Логируем артефакты шага
with mlflow.start_run(
    run_id=create_unique_run_by_name(EXPERIMENT_NAME, RUN_NAME_GET_EVENTS)
):
    mlflow.log_artifact(PATH_CLEAN_EVENTS)

### 2. Разбиение выборки

In [23]:
# Загружаем предобработанные данные
items = pd.read_parquet(PATH_CLEAN_ITEMS)
events= pd.read_parquet(PATH_CLEAN_EVENTS)

# Сортируем по времени
items.sort_values('timestamp', ignore_index=True, inplace=True)
events.sort_values('timestamp', ignore_index=True, inplace=True)

В качестве разделяющей даты выберем __1 сентября__ :)
- _строго до_ 1 сентября - обучающие события
- с 1 сенятбря _включительно_ - тестовые события

In [24]:
# Временная точка разделения обучающей/тестовой выборки
SPLIT_DATETIME = pd.to_datetime('2015-09-01')

In [25]:
# Разбиваем события на обучающую и тестовую выборки
events_train = events[events['timestamp'] < SPLIT_DATETIME]
events_test = events[events['timestamp'] >= SPLIT_DATETIME]

__NB:__ Поскольку свойства товаров в каталоге items изменяются со временем, сформируем обучающий каталог товарв items_train - состояние каталога на момент обучения модели (на 1 сентября), оставим в таблице только последние по времени записи о признаках (категории) товаров.

In [26]:
# Каталог товаров для обучения модели
items_train = (
    items[items['timestamp'] < SPLIT_DATETIME]
    .sort_values(by='timestamp', ascending=True)
    .groupby('item_id')
    .tail(1)  # оставляем только последнее значение признака
    .reset_index(drop=True)
)

Удалим из обучающей выборки события для товаров, которые отсутствуют в 'обучающем' какталоге товаров, т.е. свойства которых неизвестны на момент обучения модели

In [27]:
events_train = (
    events_train[
        events_train['item_id']
        .isin(items_train['item_id'].drop_duplicates())
    ]
    .reset_index(drop=True)
)

In [28]:
# Сохраняем таблицы локально
events_train.to_parquet(PATH_EVENTS_TRAIN)
events_test.to_parquet(PATH_EVENTS_TEST)
items_train.to_parquet(PATH_ITEMS_TRAIN)

In [29]:
# Логируем артефакты шага
with mlflow.start_run(
    run_id=create_unique_run_by_name(EXPERIMENT_NAME, RUN_NAME_TRAIN_TEST_SPLIT)
):
    mlflow.log_artifact(PATH_EVENTS_TRAIN)
    mlflow.log_artifact(PATH_EVENTS_TEST)
    mlflow.log_artifact(PATH_ITEMS_TRAIN)

### 3. Оффлайн метрики

Подготовим функции для расчета различных метрик для оффлайн рекомендаций:
- precision@K1, recall@K1 - точность и полнота для среза топ-K1 рекомендаций
- coverage@K2 - покрытие по объектам для cреза топ-K2 рекомендаций

In [30]:
def get_precision_recall_metrics(
    recs: pd.DataFrame,     
    events_test: pd.DataFrame,  
    K_prc_rec: int = 5
):
    """Calculate precision@K, recall@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - events_test: events table containing `user_id, item_id`
    - K_prc_rec: at_K value for precision and recall 
    """

    # Расчет будем вести по пользователям, удовлетворяющим двум условиям:
    # 1) для них были даны рекомендации
    # 2) они взаимодействовали с объектами в тестовый период 
    effective_users = (
        set(recs['user_id'].drop_duplicates()) 
        & set(events_test['user_id'].drop_duplicates())
    )
    events_test = (
        events_test
        .query('user_id in @effective_users')
        .reset_index(drop=True)
        .copy()
    )
    recs = (
        recs
        .query('user_id in @effective_users')
        # Оставляем только top-K рекомендаций для каждого пользователя
        .sort_values(by=['user_id', 'score'], ascending=[True, False],
                     ignore_index=True)
        .groupby('user_id')
        .head(K_prc_rec)
        .reset_index(drop=True)
        .copy()
    )

    # Помечаем события в тестовой выборке как 'ground truth'
    events_test['gt'] = True

    # Сливаем события тестовой выборки с рекомендациями
    events_recs = (
        events_test[['user_id', 'item_id', 'gt']]
        .merge(
            recs[['user_id', 'item_id', 'score']],
            on=['user_id', 'item_id'],
            how='outer'
        )
    )

    # Для НЕрелевантных рекомендации 'gt' = NaN заменяем на False 
    events_recs['gt'] = events_recs['gt'].notna()

    # Размечаем рекомендованные объекты (имеющие score) как 'rec': True
    events_recs['rec'] = events_recs['score'].notna()

    # Размечаем true positive, false positive и false negative
    events_recs['tp'] = events_recs['gt'] & events_recs['rec']
    events_recs['fp'] = ~events_recs['gt'] & events_recs['rec']
    events_recs['fn'] = events_recs['gt'] & ~events_recs['rec']

    grouped = events_recs.groupby('user_id')

    precision = (
        grouped['tp'].sum() / (grouped['tp'].sum() + grouped['fp'].sum())
    )
    precision = precision.fillna(0).mean()

    recall = (
        grouped['tp'].sum() / (grouped['tp'].sum() + grouped['fn'].sum())
    )
    recall = recall.fillna(0).mean()

    return (precision, recall)

In [31]:
def get_item_coverage_metric(
    recs: pd.DataFrame,     
    items: pd.DataFrame,    
    K_coverage: int
):
    """
    Calculate and return coverage@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - items: all items table containing `item_id` column
    - K_coverage: at_K value for coverage
    """

    # Берем срез топ-К
    top_k_recs = (
        recs
        .sort_values(by=['user_id', 'score'], ascending=[True, False],
                     ignore_index=True)
        .groupby('user_id')
        .head(K_coverage)
    )
    
    # кол-во уникальных объектов в К-срезах к общему кол-ву объектов
    return top_k_recs['item_id'].nunique() / items['item_id'].nunique()

In [32]:
def get_rec_metrics(
    recs: pd.DataFrame,     
    items: pd.DataFrame,    
    events_test: pd.DataFrame,
    K_prc_rec: int = 5,
    K_coverage: int = 50
):
    """
    Calculate and return precision@K, recall@K and coverage@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - items: all items table containing `item_id` column
    - events_test: events table containing `user_id, item_id`
    - K_prc_rec: at_K value for precision and recall 
    - K_coverage: at_K value for coverage
    """
    
    # print('Calculating precision_at_K...')
    precision_at_K, recall_at_K = (
        get_precision_recall_metrics(recs, events_test, K_prc_rec)
    )

    # print('Calculating coverage_at_K...')
    coverage_at_K = get_item_coverage_metric(recs, items, K_coverage)

    return {
        f'precision_at_{K_prc_rec}_in_percent': precision_at_K * 100,
        f'recall_at_{K_prc_rec}_in_percent': recall_at_K * 100,
        f'coverage_at_{K_coverage}_in_percent': coverage_at_K * 100
    }


### 4. Подход к построению системы, методологичекие замечания 

Вспомним, что целевые события (добавление товара в корзину) имеют очень низкую частотность.  
Посмотрим, какое кол-во пользваотелей совершивших целевое действие в тестовый период (после 1 сентябя) совершали его также в обучающий (до 1 сентября).

In [33]:
# Пользователи добавившие товар в тестовый период
effective_users_test = set(
    events_test.query('event == "addtocart"')['user_id'].drop_duplicates()
)

# Пользователи, добавлявшие товар в корзину в обучающий период
effective_users_train = set(
    events_train.query('event == "addtocart"')['user_id'].drop_duplicates()
)

# Пользователи, совершившие действие и в обучающий, и в тестовый период
effective_users_both_test_train = effective_users_test & effective_users_train

# Пользователи, добавивишие товар тестовый период
# и просматривавшие какие-либо товары в обучающий период
effective_in_test_viewed_in_train = (
    effective_users_test 
    & set(events_train.query('event == "view"')['user_id'].drop_duplicates())
)
print(f'Effective users in test  : {len(effective_users_test)}')
print('of which ')
print(f'  viewed items in train  : {len(effective_in_test_viewed_in_train)}')
print(f'  added to cart in train : {len(effective_users_both_test_train)}')

del (effective_users_test, effective_users_train,
     effective_users_both_test_train, effective_in_test_viewed_in_train)

Effective users in test  : 4275
of which 
  viewed items in train  : 527
  added to cart in train : 191


Из общего кол-во пользователей, добавлявших товары в корзину в тестовый период (4275), только ~4% (191) делали это в обучающий период.  
Таким образом, __в связи с низкой частотностью целевых событий оффлайн-история целевых действий пользователй у нас, по сути, отсутствует.__  
Единственной дополнительной информацией для построения коллаборативынх оффлайн-моделей являеются _нецелевые_ события - просмотры товаров, которые имеются тоже для очень небольшого числа пользователей, около 12%   

Исходя из изложенного, представляется разумным следующий архитектурный подход в виде объединения онлайн-рекомендаций на основе последних действий пользователя и предрассчитанных оффлайн-рекомендаций (при наличии офлайн-истории):
- онлайн рекомендации:
    - если данные об онлайн-взаимодействиях отсутсвуют (полностью холодный пользователь) - рекомендуем наиболее популярные товры (по всем категориям)
    - если есть данные о последних онлайн-взаимодействиях - рекомендуем наиболее популярные в просматриваемых категориях _плюс_ подмешиваем наиболее популярные по всем категориям
- оффлайн-рекомендации:
    - используем предварительно рассчитанные персональные рекомендации на основе имеющейся истории (при наличии)

Для дальнейшего расчета метрик выделим две группы 'тестовых' пользователей:
- `test_users_all` - все пользователи, совершившие целевое действие (добалвение в корзину) в тестовый период
- `test_users_with_history` - пользователи, совершившие целевое действие в тестовый период __И__ имеющие оффлайн историю взаимодействий (просмотр, добавление) в обучающий период

In [34]:
test_users_all = (
    events_test
    .query('event == "addtocart"')
    ['user_id']
    .drop_duplicates()
    .sort_values()
)

test_users_with_history = test_users_all[
    test_users_all.isin(
        events_train
        .query('event == "view" or event == "addtocart"')
        ['user_id']
        .drop_duplicates()
    )
]

print(f'test_users_all           : {len(test_users_all)}')
print(f'test_users_with_history  : {len(test_users_with_history)}')

test_users_all           : 4275
test_users_with_history  : 535


### 5. Построение моделей

Зафиксируем `N_RECS_USER` - число рекомендаций, которое мы будем генерировать для отдельного пользователя. 

In [35]:
# Число рекомендаций для одного пользователя
N_RECS_USER = 50

#### 5.1 Топ популярных по добавлениям в корзину

In [36]:
# Загружаем прредобработанные данные
events_train = pd.read_parquet(PATH_EVENTS_TRAIN)
events_test = pd.read_parquet(PATH_EVENTS_TEST)
items_train = pd.read_parquet(PATH_ITEMS_TRAIN)

Расчитаем индекс (score) популярности __по общему кол-ву добавлений товара в корзину__ (addtocart) различными пользователями.       
- Отнормируем от 0 до 1, разделив на максимальное число добавлений одного товаара.             
- Используем только обучающую выборку, берем N_RECS_USER наиболее популярных.

In [37]:
# Топ популярных по добавлениям в корзину
top_popular = (
    events_train
    .query('event == "addtocart"')
    .groupby('item_id')
    .agg(score=('user_id', 'count'))
    .reset_index()
    .sort_values(by='score', ascending=False, ignore_index=True)
)
top_popular['score'] /= top_popular['score'].max()
top_popular['score'] = top_popular['score'].astype('float32')

# Добавляем признак категории товара
top_popular = top_popular.merge(
    items_train[['item_id', 'category_id']],
    on='item_id',
    how='left'
)

# Визуальная проверка
top_popular.head(3)

,item_id,score,category_id
0,461686,1.00,1037
1,312728,0.59,1098
2,409804,0.57,1191


Расчитаем оффлайн метрики для всех 'тестовых' пользователей:

In [38]:
# Подготовим таблицу с рекомендациями вида (user_id, item_id, score) 
# для всех тестовых пользователей  
recs_popular_test_users_all = (
    test_users_all.to_frame()
    .merge(top_popular[['item_id', 'score']].head(N_RECS_USER), how='cross')
    .reset_index(drop=True)
)
recs_popular_test_users_all.head(1)

,user_id,item_id,score
0,155,461686,1.00


In [39]:
# Рассчитываем метрики
metrics = get_rec_metrics(
    recs_popular_test_users_all,
    items_train,
    events_test.query('event == "addtocart"')
)

# Добавляем в таблицу для сравнения
metrics_history = pd.DataFrame(metrics, index=['Top popular [all test users]'])
metrics_history

,precision_at_5_in_percent,recall_at_5_in_percent,coverage_at_50_in_percent
Top popular [all test users],0.41,1.31,0.01


И дополнительно расчитаем оффлайн метрики для 'тестовых' пользователей с оффлайн-историей:

In [40]:
# Подготовим таблицу с рекомендациями вида (user_id, item_id, score) 
# для тестовых пользователей с оффлайн-историей:
recs_popular_test_users_with_history = (
    test_users_with_history.to_frame()
    .merge(top_popular[['item_id', 'score']].head(N_RECS_USER), how='cross')
    .reset_index(drop=True)
)
recs_popular_test_users_with_history.head(1)

,user_id,item_id,score
0,155,461686,1.00


In [41]:
# Рассчитываем метрики
metrics = get_rec_metrics(
    recs_popular_test_users_with_history,
    items_train,
    events_test.query('event == "addtocart"')
)

# Добавляем в таблицу для сравнения
metrics_history = pd.concat([
    metrics_history,
    pd.DataFrame(metrics, index=['Top popular [test users with history]'])
])

metrics_history

,precision_at_5_in_percent,recall_at_5_in_percent,coverage_at_50_in_percent
Top popular [all test users],0.41,1.31,0.01
Top popular [test users with history],0.67,1.27,0.01


Неперсонализированные оффлайн-рекомендации топ-популярных товаров имеют ожидаемо невысокие метрики.

In [42]:
# Сохраняем таблицу локально
top_popular.to_parquet(PATH_TOP_POPULAR)

In [43]:
# Логируем артефакты шага
with mlflow.start_run(run_id=create_unique_run_by_name(
    EXPERIMENT_NAME, RUN_NAME_TOP_POPULAR)
):
    mlflow.log_metrics(metrics=metrics)
    mlflow.log_dict(metrics_history.to_dict(index=True),
                    'metrics_history.json')
    mlflow.log_artifact(PATH_TOP_POPULAR)

#### 5.2 Персональные ALS - наивный подход

Для пользователей имеющих оффлайн-историю, в качестве первой попытки построим персональные рекомендации методом ALS следующим 'наивным' образом:  
-  для увеличения числа 'покрытых' пользователей добавим при обучении модели (построении матрицы) к целевым взаимодействиям (добавление товара в корзину) также _нецелевые_, а именно события просмотра товаров. 

In [45]:
# Подготовим таблицу событий для построения матрицы:
# оставим только события просмора/добалвнеия в корзину,
# удалим для каждого пользователя дублирующиеся события
# взаимодействия (с тем же товаром)
events_train_als = (
    events_train
    .query('event == "addtocart" or event == "view"')
    .drop_duplicates(['user_id', 'event', 'item_id'])
    [['user_id', 'item_id', 'event']]
    .reset_index(drop=True)
)

In [46]:
# Создаем энкодеры для перекодирования идентификаторов user_id, item_id 
# в натруральный ряд  {0, 1, ...} для построения матрицы взаимодействий

user_encoder = LabelEncoder().fit(
    events_train_als['user_id'].drop_duplicates().sort_values()
)
item_encoder = LabelEncoder().fit(
    events_train_als['item_id'].drop_duplicates().sort_values()
)

In [47]:
# создаём sparse-матрицу (user x item) 
user_item_matrix = scipy.sparse.csr_matrix(
    # кортеж вида (data, (row_index, col_index))
    (
        # взаимодействие -> единица
        np.ones(len(events_train_als), dtype=np.int8),  
        (
            user_encoder.transform(events_train_als['user_id']),
            item_encoder.transform(events_train_als['item_id'])
        )
    )
)

In [48]:
# Для экономии времени - загружаем обученную модель
with open('models/als_model_naive.pkl', 'rb') as f:
    als_model = pickle.load(f)

In [49]:
# Получаем рекомендации для пользователй test_user_with_history
als_item_ids, als_scores = als_model.recommend(
    user_encoder.transform(test_users_with_history),
    user_item_matrix[user_encoder.transform(test_users_with_history)], 
    filter_already_liked_items=False,
    N=N_RECS_USER
)

In [50]:
# Перепаковываем рекомендации в таблицу формата
# (user_id, item_id, score)
personal_als = pd.DataFrame({
    'user_id': user_encoder.transform(test_users_with_history),
    'item_id': als_item_ids.tolist(),
    'score': als_scores.tolist()
})
personal_als = personal_als.explode(['item_id', 'score'], ignore_index=True)

# Приводим типы
personal_als['user_id'] = personal_als['user_id'].astype('int32')
personal_als['item_id'] = personal_als['item_id'].astype('int32')
personal_als['score'] = personal_als['score'].astype('float32')

# Перекодируем индентификаторы в исходные 
personal_als['user_id'] = (
    user_encoder.inverse_transform(personal_als['user_id'])
)
personal_als['item_id'] = (
    item_encoder.inverse_transform(personal_als['item_id'])
)
personal_als.head(1)

,user_id,item_id,score
0,155,231482,0.01


In [51]:
# Функция загрузки таблицы метрик с предыдущего шага
def get_metrics_history(experiment_name: str, run_name: str) -> pd.DataFrame:
    return pd.DataFrame(
        mlflow.artifacts.load_dict(
            get_run_by_name(experiment_name, run_name).info.artifact_uri 
            + '/metrics_history.json'
        )
    )

In [52]:
# Рассчитаем метрики
metrics = get_rec_metrics(
    personal_als,
    items_train,
    events_test.query('event == "addtocart"')
)

# Добавим в таблицу для сравнения
metrics_history = pd.concat([
    get_metrics_history(EXPERIMENT_NAME, RUN_NAME_TOP_POPULAR),
    pd.DataFrame(metrics, index=['Naive ALS [test users with history]'])
])
metrics_history

,precision_at_5_in_percent,recall_at_5_in_percent,coverage_at_50_in_percent
Top popular [all test users],0.41,1.31,0.01
Top popular [test users with history],0.67,1.27,0.01
Naive ALS [test users with history],1.23,3.67,0.68


Для пользователей, имеющих оффлайн-историю, применнеие ALS с 'наивным' добавлением к обучающим событиям _нецелевых_ событий просмотра товаров приводит к заметному увеличению метрик. 

In [53]:
# Сохраняем таблицу локально
personal_als.to_parquet(PATH_NAIVE_ALS)

# Логируем артефакты шага
with mlflow.start_run(run_id=create_unique_run_by_name(
    EXPERIMENT_NAME, RUN_NAME_NAIVE_ALS)
):
    mlflow.log_metrics(metrics=metrics)
    mlflow.log_dict(metrics_history.to_dict(index=True),
                    'metrics_history.json')
    mlflow.log_artifact(PATH_NAIVE_ALS)

#### 5.3 Персональные ALS - взвешивание обучающих событий

Попробуем улучшить качество персональных ALS рекомендаций.   
Для этого будем добавлять события в матрицу взаимодействий с разными весами:
 - более _низкий_ вес для событий _просмотра_ товара
 - более _высокий_ вес для событий _добавления_ тоавара в корзину

__NB__ Используем следующую эвристику: учитывая, что медианное число просмотров товара перед добавлением в корзину равно 3 (трем), т.е. соотношение _частот_ просмотра/добавления равно 3 : 1 (для 'целевых' пользователей) - развернем пропорцию и установим ссотношение _весов_ для событий просмотра/добалвения в матрице взаимодействия как __1 : 3__.

In [54]:
# Веса вестов для различных типов событий 
VIEW_WEIGHT = .25
ADDTOCART_WEIGHT = .75

# Добваляем столбец с весами 
# NB: в таблице уже оставлено только два типа событий (view и addtocart)
events_train_als['weight'] = events_train_als['event'].map(
    lambda x: VIEW_WEIGHT if x == 'view' else ADDTOCART_WEIGHT
).astype('float32')

In [55]:
# Создаем энкодеры для перекодирования идентификаторов user_id, item_id 
# в натруральный ряд  {0, 1, ...} для построения матрицы взаимодействий

user_encoder = LabelEncoder().fit(
    events_train_als['user_id'].drop_duplicates().sort_values()
)
item_encoder = LabelEncoder().fit(
    events_train_als['item_id'].drop_duplicates().sort_values()
)

In [56]:
# создаём sparse-матрицу (user x item) 
user_item_matrix = scipy.sparse.csr_matrix(
    # кортеж вида (data, (row_index, col_index))
    (   
        # Используем разные веса для разных событий
        events_train_als['weight'].to_numpy(),  
        (
            user_encoder.transform(events_train_als['user_id']),
            item_encoder.transform(events_train_als['item_id'])
        )
    )
)

In [57]:
# Для экономии времени загружаем рассчитанные рекомендации
personal_als = pd.read_parquet(PATH_WEIGHTED_ALS)

In [58]:
# Рассчитаем метрики
metrics = get_rec_metrics(
    personal_als[personal_als['user_id'].isin(test_users_with_history)],
    items_train,
    events_test.query('event == "addtocart"')
)

# Добавим в таблицу для сравнения
metrics_history = pd.concat([
    get_metrics_history(EXPERIMENT_NAME, RUN_NAME_NAIVE_ALS),
    pd.DataFrame(metrics, index=['Weighted ALS [test users with history]'])
])
metrics_history

,precision_at_5_in_percent,recall_at_5_in_percent,coverage_at_50_in_percent
Top popular [all test users],0.41,1.31,0.01
Top popular [test users with history],0.67,1.27,0.01
Naive ALS [test users with history],1.23,3.67,0.68
Weighted ALS [test users with history],1.40,4.07,1.07


Взвешивание событий при построении матрицы взаимодействий приводит к заметному улучшению оффлайн-метрик (на 10% по метрикам precision/recall). 

In [59]:
# Сохраняем таблицу локально
personal_als.to_parquet(PATH_WEIGHTED_ALS)

# Логируем артефакты шага
with mlflow.start_run(run_id=create_unique_run_by_name(
    EXPERIMENT_NAME, RUN_NAME_WEIGHTED_ALS)
):
    mlflow.log_metrics(metrics=metrics)
    mlflow.log_dict(metrics_history.to_dict(index=True),
                    'metrics_history.json')
    mlflow.log_artifact(PATH_WEIGHTED_ALS)

### 5. Выводы

__Действия по очистке (фильтрации) даыннх по результатам EDA__

- из каталога товаров удаляем товары с item_id, отсутствующим в дереве категорий
- из всех признаков товаров, оставляем только category_id
- из таблицы событий удаляем события, относящиеся к 'неизвестынм' товарам (отсутствующим в каталоге items)

__Разделение выборки__
- в качестве разделяющей даты выберем __1 сентября__ :)
- поскольку свойства товаров в каталоге items изменяются со временем, сформируем обучающий каталог товарв items_train - состояние каталога на момент обучения модели (на 1 сентября), оставим в таблице только последние по времени записи о признаках (категории) товаров.
- удалим из обучающей выборки события для товаров, которые отсутствуют в 'обучающем' какталоге товаров, т.е. свойства которых неизвестны на момент обучения модели

__Метрики__

Для оценки качества оффлайн рекомендаций будем исопльзовать следующие метрики:
- _precision@5, recall@5_ - точность и полнота для среза топ-5 рекомендаций
- _coverage@50_ - покрытие по объектам для cреза топ-50 рекомендаций

__Подход к построению системы, методологичекие замечания__

Из общего кол-во пользователей, добавлявших товары в корзину в тестовый период (4275), только ~4% (191) делали это в обучающий период.  
__В связи с низкой частотностью целевых событий оффлайн-история целевых действий пользователй у нас, по сути, отсутствует.__  
Единственной дополнительной информацией для построения коллаборативынх оффлайн-моделей являеются _нецелевые_ события - просмотры товаров, которые имеются тоже для очень небольшого числа пользователей (около 12%, оценка по тестовой выборке)   

Исходя из изложенного, представляется разумным следующий архитектурный подход в виде объединения онлайн-рекомендаций на основе последних действий пользователя и предрассчитанных персональных оффлайн-рекомендаций (для ~12% пользователй с оффлайн-историей):
- онлайн рекомендации:
    - если данные об онлайн-взаимодействиях отсутсвуют (полностью холодный пользователь) - рекомендуем наиболее популярные товры (по всем категориям)
    - если есть данные о последних онлайн-взаимодействиях - рекомендуем наиболее популярные в просматриваемых категориях _плюс_ подмешиваем наиболее популярные по всем категориям
- оффлайн-рекомендации:
    - используем предварительно рассчитанные персональные рекомендации на основе имеющейся истории (при наличии)


__Эксперименты с различными оффлайн-моделями__

- _Топ популярных_: Неперсонализированные оффлайн-рекомендации топ-популярных товаров имеют ожидаемо невысокие метрики.

- _Naive ALS_: Для пользователей, имеющих оффлайн-историю, применнеие ALS с '_наивным_' добавлением к обучающим событиям _нецелевых_ событий просмотра товаров приводит к заметному увеличению метрик. 

- _Weighted ALS_ Взвешивание событий при построении матрицы взаимодействий приводит к заметному улучшению оффлайн-метрик (на 10% по метрикам precision/recall).

В связи с малой долей пользователей с оффлайн-историей и ограниченным временм на выполение проекта, дальнейшее улучшение качества персональных оффлайн рекомендаций (разработка дополнитлеьных кандидатов, построение ранжирующей модели и т.д.) представлеется излишней.

Итоговая таблица с оффлайн-метриками:

<div>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>precision_at_5_in_percent</th>
      <th>recall_at_5_in_percent</th>
      <th>coverage_at_50_in_percent</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Top popular [all test users]</th>
      <td>0.41</td>
      <td>1.31</td>
      <td>0.01</td>
    </tr>
    <tr>
      <th>Top popular [test users with history]</th>
      <td>0.67</td>
      <td>1.27</td>
      <td>0.01</td>
    </tr>
    <tr>
      <th>Naive ALS [test users with history]</th>
      <td>1.23</td>
      <td>3.67</td>
      <td>0.68</td>
    </tr>
    <tr>
      <th>Weighted ALS [test users with history]</th>
      <td>1.40</td>
      <td>4.07</td>
      <td>1.07</td>
    </tr>
  </tbody>
</table>
</div>

В качестве итоговых моделей для продуктивизации будем использовать _Топ популярных_ (для онлайн-рекомендаций, в т.ч. с учетом категории для персонифицированных онлайн) и _Weighted ALS_ для персонифицированных оффлайн. 

